### Create and Update Predict-Log File
The case-study will demonstrate how logging is tied to a machine learning model to enable performance monitoring.

In [1]:
## import the libraries
import time,os,re,csv,sys,uuid,joblib
from datetime import date
import numpy as np
from sklearn import svm
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import os
import joblib
import time
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, f1_score
from sklearn.svm import SVC
import imblearn.pipeline as pl
from imblearn.pipeline import make_pipeline
    
plt.style.use('seaborn')
%matplotlib inline

Using TensorFlow backend.


In [2]:
## load the data that we have already done
df = pd.read_csv(os.path.join(".",r"data/aavail-target.csv"))

In [3]:
## pull out the target and remove uneeded columns
_y = df.pop('is_subscriber')   # drop (the column) and return the values from it
y = np.zeros(_y.size)          # form a equal size array "y"
y[_y==0] = 1                   # convert "y" as churn-data ("=1")

## then drop the other columns as un-needed
df.drop(columns=['customer_id','customer_name'],inplace=True)
df.head()

,country,age,subscriber_type,num_streams
0,united_states,21,aavail_premium,23
1,singapore,30,aavail_unlimited,12
2,united_states,21,aavail_premium,22
3,united_states,20,aavail_basic,19
4,singapore,21,aavail_premium,23


In [4]:
def train_model(X,y,saved_model):
    """
    function to train model
    """ 
    X = df
    
    ## Perform a train-test split
    rs = random_state = 1
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, 
                                                stratify=y, random_state=rs)
    ## Specify parameters and model
    ## for numeric
    numeric_features = ['age', 'num_streams']
    numeric_transformer = Pipeline(steps=[
                         ('imputer', SimpleImputer(strategy='mean')),
                         ('scaler', StandardScaler())])

    ## for categorical
    categorical_features = ['country', 'subscriber_type']
    categorical_transformer = Pipeline(steps=[
      ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
      ('onehot', OneHotEncoder(handle_unknown='ignore'))])

    ## define "preprocessor" and passing it the specific values
    preprocessor = ColumnTransformer(
      transformers=[
                   ('num', numeric_transformer, numeric_features),
                   ('cat', categorical_transformer, categorical_features)])

    ## fit model on training data
    ## (i) svm
    param_grid_svm = {
                    'svm__C': [0.01,0.1,0.5,1.0,1.5,5.0,10.0],
                    'svm__gamma': [0.001,0.01,0.1]
    }
    best_params = {}
    pipe_svm = Pipeline(steps=[('pre', preprocessor),
                                ('svm',SVC(kernel='rbf',
                                           class_weight='balanced'))])
    grid = GridSearchCV(pipe_svm, param_grid=param_grid_svm, cv=5, 
                       iid=False, n_jobs=-1)
    grid.fit(X_train, y_train)
    y_pred = grid.predict(X_test)

    print("-->".join(pipe_svm.named_steps.keys()))

    best_params = grid.best_params_
    #print("best parameters:", best_params)
    print("f1_score",round(f1_score(y_test, y_pred,average='binary'),3))

    ## retrain using all data
    grid.fit(X, y)
    print("... saving model: {}".format(saved_model))
    joblib.dump(grid,saved_model)

In [5]:
def _update_predict_log(y_pred,y_prob,query,runtime):
    """
    update predict log file
    """
    ## name the logfile using something that cycles with date (day, month, year)    
    today = date.today()
    logfile = "Prediction-{}-{}.log".format(today.year, today.month)

    ## write the data to a csv file    
    header = ['unique_id','timestamp','y_pred','y_prob','x_shape','model_version','runtime']
    write_header = False
    if not os.path.exists(logfile):
        write_header = True
    with open(logfile,'a') as csvfile:
        writer = csv.writer(csvfile, delimiter=',', quotechar='|')
        if write_header:
            writer.writerow(header)

        to_write = map(str,[uuid.uuid4(),time.time(),y_pred,y_prob,query.shape,MODEL_VERSION,runtime])
        writer.writerow(to_write)

In [6]:
def model_load():
    """
    funtion to load model
    """

    if not os.path.exists(saved_model):
        raise Exception("Model '{}' cannot be found did you train the model?".format(saved_model))
    
    model = joblib.load(saved_model)
    return(model)

In [7]:
def predict(query):
    """
    generic function for prediction
    """

    ## start timer for runtime
    time_start = time.time()
    
    ## ensure the model is loaded
    model = joblib.load(saved_model)

    ## output checking
    if len(query.shape) == 1:
        query = query.reshape(1, -1)
    
    ## make prediction and gather data for log entry
    y_pred = model.predict(query)
    y_prob = None
    if 'predict_proba' in dir(model) and model.probability == True:
        y_proba = model.predict_proba(query)
    m, s = divmod(time.time()-time_start, 60)
    h, m = divmod(m, 60)
    runtime = "%03d:%02d:%02d"%(h, m, s)

    ## update the log file
    _update_predict_log(y_pred,y_proba,query,runtime)
    
    return(y_pred)